In [1]:
# !pip install -qU \
#     langchain==0.3.23 \
#     langchain-community==0.3.21 \
#     langchain-pinecone==0.2.5 \
#     langchain-openai==0.3.12 \
#     datasets==3.5.0

In [32]:

# from langchain.chat_models import ChatOpenAI
# from langchain.schema import SystemMessage, HumanMessage, AIMessage

# chat = ChatOpenAI(
#     openai_api_key="sk-669dc1afbce54ccaaecdd27b320777fd",
#     openai_api_base="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
#     model="qwen-turbo"
# )

# messages = [
#     SystemMessage(content="You are a helpful assistant."),
#     HumanMessage(content="Hi AI, how are you today?"),
#     AIMessage(content="I'm great thank you. How can I help you?"),
#     HumanMessage(content="I'd like to understand string theory.")
# ]

# response = chat(messages)
# print(response.content)


String Theory is a complex and abstract concept in physics that attempts to reconcile quantum mechanics with general relativity. It proposes that the fundamental constituents of the universe are not point-like particles, but rather one-dimensional "strings" - basically tiny loops of string.

These strings vibrate at different frequencies, and these vibrations are what create the various particles that make up the universe, such as electrons, photons, and quarks. 

In essence, String Theory posits that everything in the universe is made up of these tiny strings, and the properties of particles (like mass and charge) are determined by the way these strings vibrate. Different vibration patterns correspond to different particle types.

The theory also predicts the existence of extra dimensions beyond the three we experience daily (length, width, height), which are compactified or rolled up at very small scales.

String theory is an active area of research in theoretical physics, but it's i

In [107]:
from openai import OpenAI
import os
from tqdm import tqdm
import numpy as np
from dashvector import Client as DashClient, Doc

# Step 1: Setup DashScope OpenAI-compatible client
client = OpenAI(
    api_key="sk-91d59f4cf5ff44cf9f8280a91dacbb1c",
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)


dashvector_client = DashClient(
    api_key="sk-eAO7D10gZP2kzhjeKmiI7b6bOHTmoD6B9CAC6318F11F097B99A92126D810D",
    endpoint="vrs-sg-l0z49hq250003n.dashvector.ap-southeast-1.aliyuncs.com"
)

In [108]:
query = "What is the coverage limit for Third-Party liability under Comprehensive Cover?"

response = client.embeddings.create(
    model="text-embedding-v3",
    input=query,
    encoding_format="float"
)
query_embedding = response.data[0].embedding  # Extract the embedding
query_embedding = np.array(response.data[0].embedding)
query_embedding = query_embedding / np.linalg.norm(query_embedding)  # L2 normalize
# Ensure embeddings are lists of floats (not arrays)
query_embedding = [float(x) for x in query_embedding]  # Convert numpy array to list

In [109]:
# Step 2: Search DashVector for top 5 similar docs
collection = dashvector_client.get(name="quickstart")
search_results = collection.query(query_embedding, topk=10)  # Increase topk to 10

In [110]:
# Step 3: Extract retrieved chunks (assuming `fields` exist in result docs)
retrieved_chunks = [doc.fields["text"] for doc in search_results]

# Step 4: Prepare prompt for Qwen with retrieved context + question
context_text = "\n---\n".join(retrieved_chunks)

prompt = f"""You are an AI assistant specialized in answering questions about insurance policies.
Your task is to provide accurate answers based solely on the provided context from the policy document.
If the context does not contain the answer, respond with "The information is not available in the provided documents.

Based on the following context:

{''.join([f'[{j+1}] {chunk}' for j, chunk in enumerate(retrieved_chunks)])}

Answer the question:
{query}

Instructions:
- Use only the context above to formulate your answer.
- If the answer is unclear, state that it is not found in the provided context.
"""

# Step 5: Call Qwen model for completion
completion = client.chat.completions.create(
    model="qwen-turbo",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
    max_tokens=512,
)

print(completion.choices[0].message.content)


The coverage limit for Third-Party liability under Comprehensive Cover is up to MYR 1,000,000 per claim.
